In [4]:
from dotenv import load_dotenv
import os

load_dotenv()
print(os.environ.get('BIGQUERY_PROJECT_NAME'))

transfusion-reactions


In [6]:
from google.cloud import bigquery

client = bigquery.Client(os.environ.get('BIGQUERY_PROJECT_NAME'))
query = """WITH first_admission_time AS
(
  SELECT
      p.subject_id, p.dob, p.gender
      , MIN (a.admittime) AS first_admittime
      , MIN( DATETIME_DIFF(admittime, dob, YEAR) )
          AS first_admit_age
  FROM `physionet-data.mimiciii_clinical.patients` p
  INNER JOIN `physionet-data.mimiciii_clinical.admissions` a
  ON p.subject_id = a.subject_id
  GROUP BY p.subject_id, p.dob, p.gender
  ORDER BY p.subject_id
)
, age as
(
  SELECT
      subject_id, dob, gender
      , first_admittime, first_admit_age
      , CASE
          -- all ages > 89 in the database were replaced with 300
          WHEN first_admit_age > 89
              then '>89'
          WHEN first_admit_age >= 14
              THEN 'adult'
          WHEN first_admit_age <= 1
              THEN 'neonate'
          ELSE 'middle'
          END AS age_group
  FROM first_admission_time
)
select age_group, gender
  , count(subject_id) as NumberOfPatients
from age
group by age_group, gender"""
results = client.query(query).to_dataframe()
print(results)

  age_group gender  NumberOfPatients
0       >89      F              1294
1     adult      F             15476
2   neonate      F              3629
3     adult      M             21179
4       >89      M               697
5   neonate      M              4245


/home/kmorhun/.pyenv/versions/3.12.7/envs/transfusion/lib/python3.12/site-packages/google/cloud/bigquery/table.py:1727: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(
